In [1]:
import jraph
import jax
import jax.numpy as jnp
import e3nn_jax as e3nn
import numpy as np


import sys
sys.path.append('..')


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2
import train
import models
import datatypes
import analyses.generate_plots as generate_plots
import input_pipeline_tf
import configs.mace as mace
import analyses.analysis as analysis

In [4]:
basedir = "../potato_workdirs/extras/sample_complexity"
results = analysis.get_results_as_dataframe(generate_plots.ALL_MODELS, generate_plots.ALL_METRICS, basedir)
results['val'][results['val']['model'] == 'mace'].sort_values(by='max_l')

,config.activation,config.avg_num_neighbors,config.compute_padding_dynamically,config.eval_every_steps,config.focus_predictor.latent_size,config.focus_predictor.num_layers,config.learning_rate,config.learning_rate_schedule,config.learning_rate_schedule_kwargs.decay_steps,config.learning_rate_schedule_kwargs.init_value,...,config.val_molecules,total_loss,position_loss,focus_loss,atom_type_loss,model,max_l,num_params,num_train_molecules,config.cutoff
22,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,5.531107,4.371337,0.274245,0.885521,mace,0,130464,11904,NaN
4,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,5.343208,4.364789,0.261435,0.716976,mace,0,130464,23808,NaN
16,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,5.283570,4.341050,0.245607,0.696912,mace,0,130464,47616,NaN
28,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,5.799916,4.388112,0.331969,1.079842,mace,0,130464,5952,NaN
10,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,5.950015,4.450755,0.339231,1.160030,mace,0,130464,2976,NaN
29,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,3.871002,2.602349,0.308156,0.960490,mace,1,207648,5952,NaN
23,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,3.790412,2.643588,0.278330,0.868498,mace,1,207648,11904,NaN
5,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,3.290397,2.314846,0.270049,0.705504,mace,1,207648,23808,NaN
17,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,3.202348,2.282289,0.254026,0.666040,mace,1,207648,47616,NaN
11,softplus,15.0,False,1000,128,2,0.001,sgdr,1000,0.001,...,47616,4.562701,3.086997,0.311900,1.163811,mace,1,207648,2976,NaN


In [ ]:
results['val'].to_clipboard()

In [ ]:
def eval():
    for x in datasets["val"].take(10).as_numpy_iterator():
        mask = jraph.get_graph_padding_mask(x)
        print(x.n_node[mask].sum())

eval()


In [33]:
import tensorflow as tf

ds = tf.data.Dataset.range(10)

In [38]:
for x in ds.take(3).as_numpy_iterator():
    print(x)

0
1
2


In [8]:
config = mace.get_config()
datasets = input_pipeline_tf.get_raw_qm9_datasets(config)
for x in datasets["val"].take(10).as_numpy_iterator():
    print(x['target_positions'].sum())

-1.4348999
0.0
1.0533
0.0
-1.18
0.26699996
-0.75749993
-1.0186
-1.3825
-0.56520003


In [57]:
ds = datasets['val'].map(
    input_pipeline_tf._convert_to_graphstuple,
    num_parallel_calls=tf.data.AUTOTUNE,
    deterministic=True,
)

ds_b = jraph.dynamically_batch(graphs_tuple_iterator=iter(ds),
                                n_node=config.max_n_nodes,
                                n_edge=config.max_n_edges,
                                n_graph=config.max_n_graphs)

36
92
148
116
104
86
106
90
143
115
112


In [72]:
for step, x in enumerate(ds_b):
    mask = jraph.get_graph_padding_mask(x)
    print(x.n_node[mask].sum())
    if step == 10:
        break

125
130
80
130
132
123
118
103
110
104
126


In [105]:
import functools

example_graph = next(ds.as_numpy_iterator())
example_padded_graph = jraph.pad_with_graphs(
    example_graph, n_node=config.max_n_nodes, n_edge=config.max_n_edges, n_graph=config.max_n_graphs
)
padded_graphs_spec = input_pipeline_tf._specs_from_graphs_tuple(example_padded_graph)

# Batch and pad each split separately.
batching_fn = functools.partial(
    jraph.dynamically_batch,
    graphs_tuple_iterator=iter(ds),
    n_node=config.max_n_nodes,
    n_edge=config.max_n_edges,
    n_graph=config.max_n_graphs,
)
ds_tf = tf.data.Dataset.from_generator(
    batching_fn, output_signature=padded_graphs_spec
)
ds_tf = ds_tf.take(100).cache()

In [109]:
for step, x in enumerate(ds_tf.as_numpy_iterator()):
    mask = jraph.get_graph_padding_mask(x)
    print(x.n_node[mask].sum())
    if step == 10:
        break

56
109
129
106
90
84
63
66
104
134
104


2023-04-04 20:03:48.126544: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [9]:
# Mimic what we do in train.py.
config = mace.get_config()

config.max_n_graphs = 32
rng = jax.random.PRNGKey(0)
rng, dataset_rng = jax.random.split(rng)

# Obtain graphs.
datasets = input_pipeline_tf.get_datasets(dataset_rng, config)
train_iter = datasets["train"].as_numpy_iterator()
init_graphs = next(train_iter)

# Set up dummy variables to obtain the structure.
rng, init_rng = jax.random.split(rng)
net = train.create_model(config, run_in_evaluation_mode=False)
params = jax.jit(net.init)(init_rng, init_graphs)

#ds = ds.shuffle(buffer_size=5, reshuffle_each_iteration=False)

In [22]:
for x in datasets["train_eval_final"].as_numpy_iterator():
    mask = jraph.get_graph_padding_mask(x)
    print(x.n_node[mask].sum())

85
146
117
132
114
119
109
105
89
102
127
122
112
107
120
105
111
135
120
135
125
141
120
115
134
134
179
149
134
116
90
130
143
138
99
113
103
103
141
116
155
90
112
83
118
129
112
102
105
123
91
107
146
117
116
121
151
131
133
128
111
89
139
154
97
113
92
114
114
109
92
108
104
144
86
98
139
125
114
133
137
141
123
159
124
114
89
170
129
72
73
149
103
123
114
119
137
164
129
123
103
105
131
124
147
90
125
164
166
152
129
156
138
126
118
129
167
130
144
110
103
119
106
147
100
98
108
92
136
109
124
99
120
102
152
148
103
142
134
89
130
137
107
112
165
102
120
130
127
148
136
153
139
151
123
169
138
106
147
136
127
144
162
152
147
141
168
103
166
107
117
133
126
150
99
141
139
127
140
94
143
131
153
160
190
133
127
99
96
140
170
158
99
104
169
100
123
125
117
135
113
143
137
105
114
171
158
140
178
144
119
101
141
94
118
86
138
104
118
143
141
165
89
148
108
131
130
145
161
141
114
128
140
135
138
138
122
130
149
130
138
164
155
135
121
144
107
130
130
129
142
98
146
155
152
115
113
13

In [94]:
import tensorflow as tf
ds = tf.data.Dataset.from_tensor_slices([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
ds = ds.shuffle(buffer_size=5, reshuffle_each_iteration=False)

In [95]:
for x in ds.as_numpy_iterator():
    print(x)

2
4
1
6
3
0
5
9
8
7


In [ ]:
print("padding")
print("nodes", config.max_n_nodes)
print("edges", config.max_n_edges)
print("graphs", config.max_n_graphs)

In [ ]:
count = 0
for step, graphs in enumerate(datasets["train"].as_numpy_iterator()):
    if step % 1000 == 1:
        print(step, count / step)
    if step == 10000:
        break

    graphs = jax.tree_map(jnp.array, graphs)
    graphs = datatypes.Fragments.from_graphstuple(graphs)
    count += jraph.get_graph_padding_mask(graphs).sum()

print(count / step)


In [ ]:
graphs = jraph.GraphsTuple(
    nodes=jnp.array([[1, 2, 3], [1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    edges=jnp.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    globals=jnp.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]]),
    n_node=jnp.array([2, 1, 1]),
    n_edge=jnp.array([1, 1, 1]),
    senders=jnp.array([0, 0, 0]),
    receivers=jnp.array([0, 0, 0]),
)

In [ ]:
num_graphs = 4
rng, *species_rngs = jax.random.split(jax.random.PRNGKey(0), num_graphs + 1)

target_species_probs = jnp.ones((num_graphs, models.NUM_ELEMENTS)) / models.NUM_ELEMENTS
target_species = jax.vmap(lambda key, p: jax.random.choice(
    key, models.NUM_ELEMENTS, p=p))((species_rngs,), target_species_probs)


In [ ]:
e3nn.IrrepsArray("10x0e", jnp.arange(10))

In [ ]:

myhost

In [ ]:
# Get results.
results = analysis.get_results_as_dataframe(generate_plots.ALL_MODELS, generate_plots.ALL_METRICS, "../potato_workdirs/v4/")

In [ ]:
results['val'].sort_values('total_loss')

In [ ]:
import re
seed, start, end = [int(val) for val in re.findall(r'\d+', "fragments_seed01_from130944_to133920")]
start, end

In [ ]:
def segment_sample(probabilities, segment_ids, num_segments, rng):
    """Sample from a categorical distribution across each segment.
    Args:
        segment_ids: A 1D array of segment ids.
        probs: A 1D array of probabilities.
    Returns:
        A 1D array of samples.
    """
    def sample_for_segment(rng, i):
        return jax.random.choice(rng, node_indices, p=jnp.where(i == segment_ids, probabilities, 0.))
    
    node_indices = jnp.arange(len(segment_ids))
    rngs = jax.random.split(rng, num_segments)
    return jax.vmap(sample_for_segment)(rngs, jnp.arange(num_segments))

In [ ]:
focus_logits = graphs.nodes.sum(axis=1)
probs = jraph.partition_softmax(focus_logits, graphs.n_node)
print(probs)
for seed in range(100):
    print(segment_sample(probs, jnp.asarray([0, 0, 1, 2]), 3, jax.random.PRNGKey(seed)))

In [ ]:
nodes = jnp.asarray([2, 3, 1, 4])
segment_max = e3nn.scatter_max(nodes, nel=graphs.n_node)
segment_max_expanded = e3nn.scatter_max(nodes, map_back=True, nel=graphs.n_node)
print(segment_max_expanded)
# segment_max_expanded = jnp.asarray([segment_max[0], segment_max[0], segment_max[1], segment_max[2]])

expected = 0 + jnp.log(1 + e3nn.scatter_sum(jnp.exp(nodes - 0), nel=graphs.n_node))
computed = segment_max + jnp.log(jnp.exp(-segment_max) + e3nn.scatter_sum(jnp.exp(nodes - segment_max_expanded), nel=graphs.n_node))

expected, computed

In [ ]:
import analysis


In [ ]:
analysis.load_from_workdir("/Users/ameyad/Documents/spherical-harmonic-net/potato_workdirs/workdirs/mace/interactions=1/l=0/channels=32")

In [ ]:
lmax = 3
irreps = e3nn.Irreps(e3nn.Irrep.iterator(lmax))
e3nn.IrrepsArray(irreps=irreps, array=jnp.ones((10, irreps.dim)))

In [ ]:
e3nn.scatter_sum(data=graphs.nodes, nel=graphs.n_node)